# eEDB-011-2024-3

## Atividade 4 – Ingestão e ETL com linguagem de programação (Python + Spark + SQL)

* Utilizar linguagem de programação Python para ingestão e tratamento de dados. Para processo de transformação
deve ser realizado via SQL com uma das seguintes ferramentas:
  * DBT (indicada)
  *  SQL Mash – alternativa
  *  Coalesce - Alternativa

*  Base final deve ser um banco de dados relacional já utilizado nos exercícios anteriores.
* O processamento SQL deve ser utilizado via ferramenta mais uma engine de SQL que pode ser o próprio Banco de
Dados bem como um banco de dados em memória como Duckdb ou SLQLite, ou qualquer outra alternativa.
*  Gerar uma tabela final com os dados tratados e unidos.
  * O tratamento de dados deve ser realizado através da ferramenta escolhida para SQL
*  Adicionar as seguintes camadas de processamento, dentro do próprio banco de dados ou em disco local. A
Camada Delivery deve obrigatoriamente ter estar também no formato de uma tabela final dentro do banco de
dados relacional:
  *  RAW – formato dos dados livre
  *  Trusted – formato de dados em Parquet ou ORC or AVRO (indicado Parquet)
  *  Delivery– formato de dados em Parquet ou ORC or AVRO (indicado Parquet)

- **Grupo 02**:
    - Aline Bini
    - Ana Lívia Franco
    - Ana Priss
    - João Squinelato
    - Marcelo Pena
    - Thais Siqueira

- [Github](https://github.com/Squinelato/eEDB-011-2024-3 "eEDB-011-2024-3")

```Ingestão De Dados | Agosto 2024```

## To Do

- raw
- trusted
- delivery

In [1]:
%pip install pyspark
%pip install unidecode 
%pip install findspark

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lpad, col, lpad, concat, sha1, regexp_replace, udf, lower, lit, when
from pyspark.sql.types import StringType, FloatType, IntegerType, StructType, StructField, ArrayType
from unidecode import unidecode

import findspark
import os

In [4]:
findspark.init()

In [5]:
spark = SparkSession.builder \
    .master('local') \
    .appName('Basic ETL') \
    .config('spark.executor.timeout', "1200s") \
    .config('spark.sql.broadcastTimeout', '1200s') \
    .config('spark.rpc.askTimeout', '600s') \
    .config('spark.executor.heartbeatInterval', '120s') \
    .config('spark.network.timeout', '1200s') \
    .getOrCreate()

In [6]:
spark

---
## **Raw**

### **Banks file**

In [7]:
# banks_csv_path = '../Fonte de Dados/Bancos/EnquadramentoInicia_v2.tsv'

#drive
banks_csv_path = '/content/drive/MyDrive/USP/3 - Ingestão de dados/Atividade4/Fonte de Dados/Bancos/EnquadramentoInicia_v2.tsv'

# Leitura do arquivo CSV e criação da view temporária
bank_df = spark.read.csv(banks_csv_path, sep='\t', encoding='utf8', header=True)
bank_df.createOrReplaceTempView("bank_temp")
rwzd_bank = spark.sql("SELECT * FROM bank_temp")

Analisando o esquema dos dados

In [8]:
spark.sql("DESCRIBE bank_temp").show(truncate=False)

+--------+---------+-------+
|col_name|data_type|comment|
+--------+---------+-------+
|Segmento|string   |NULL   |
|CNPJ    |string   |NULL   |
|Nome    |string   |NULL   |
+--------+---------+-------+



Contando a quantidade de linhas

In [9]:
total_records = spark.sql("SELECT COUNT(*) AS total FROM bank_temp").collect()[0]['total']
print(f"Total de registros: {total_records}")

Total de registros: 1474


Salvando dados na camada _raw_ no formato parquet.

<sub><i>Usando o Google Colab, o caminho ./raw/bank/ se refere ao sistema de arquivos temporário da máquina virtual onde o notebook está sendo executado.
No Colab, esse caminho fica na raiz da instância, mas o armazenamento é temporário. Isso significa que, uma vez que a sessão do Colab seja finalizada, os arquivos serão perdidos.<i><sub>

Outra abordagem é salvar um DataFrame Spark tanto em formato .parquet quanto em .csv diretamente no Google Drive:

In [10]:
# Drive
drive_path = '/content/drive/MyDrive/USP/3 - Ingestão de dados/Atividade4/raw/banks'

# Criar a pasta
os.makedirs(drive_path, exist_ok=True)

parquet_path = os.path.join(drive_path, 'rwzd_bank.parquet')
csv_path = os.path.join(drive_path, 'rwzd_bank.csv')

# Salvar o DataFrame como .parquet
rwzd_bank.write.mode("overwrite").parquet(parquet_path)

# Salvar o DataFrame como .csv (com cabeçalho)
# ! Salvei todos os arquivos também como csv, para verificar os resultados, mas
# isso pode ser deletado
rwzd_bank.write.mode("overwrite").option("header", "true").csv(csv_path)

print(f"Arquivos salvos em: {drive_path}")


Arquivos salvos em: /content/drive/MyDrive/USP/3 - Ingestão de dados/Atividade4/raw/banks


### **Employees file**

Localizando todos os arquivos contendo dados de empregados

In [11]:
employee_dir = '/content/drive/MyDrive/USP/3 - Ingestão de dados/Atividade4/Fonte de Dados'
employee_files = os.listdir(employee_dir)
employee_paths = list(map(lambda file: os.path.join(employee_dir, file), employee_files))[1]
employee_paths

'/content/drive/MyDrive/USP/3 - Ingestão de dados/Atividade4/Fonte de Dados/Empregados'

Lendo todos os arquivos de empregados como um único conjunto de dados


<sub><i>Usei o unionByName do spark para unir os dois dataFrames considerando os nomes das colunas. Isso significa que, mesmo que as colunas estejam em ordens diferentes ou que um dos dataFrames tenha colunas extras, ele vai alinhar os dados corretamente com base nos nomes das colunas.
allowMissingColumns=True: Quando definido como True, essa opção permite que a união ocorra mesmo se um dos dataFrames tiver colunas extras que o outro não possui. Nessas colunas faltantes, a intenção é preencher os valores com null.<i><sub>

In [12]:
# Lendo os dois csv's
df1 = spark.read.csv("/content/drive/MyDrive/USP/3 - Ingestão de dados/Atividade4/Fonte de Dados/Empregados/glassdoor_consolidado_join_match_less_v2.csv", sep='|',header=True, inferSchema=True)
df2 = spark.read.csv("/content/drive/MyDrive/USP/3 - Ingestão de dados/Atividade4/Fonte de Dados/Empregados/glassdoor_consolidado_join_match_v2.csv", sep='|',header=True, inferSchema=True)

employee_df = df1.unionByName(df2, allowMissingColumns=True)


In [13]:
employee_df.createOrReplaceTempView("employee_temp")
spark.sql("SELECT * FROM employee_temp LIMIT 100").show()

+--------------------+-------------+-------------+--------------+--------------+--------------------+---------------------+----------------+--------------------+--------------------+--------------------+-----+-----------------+----------------------+-----------------+--------------+------------------------+-------------------------+---------------------------------+----------------------------------+--------+--------------------+-------------+--------+
|       employer_name|reviews_count|culture_count|salaries_count|benefits_count|    employer-website|employer-headquarters|employer-founded|   employer-industry|    employer-revenue|                 url|Geral|Cultura e valores|Diversidade e inclusão|Qualidade de vida|Alta liderança|Remuneração e benefícios|Oportunidades de carreira|Recomendam para outras pessoas(%)|Perspectiva positiva da empresa(%)|    CNPJ|                Nome|match_percent|Segmento|
+--------------------+-------------+-------------+--------------+--------------+------

In [14]:
total_records_employee = spark.sql("SELECT COUNT(*) AS total FROM employee_temp").collect()[0]['total']
print(f"Total de registros: {total_records_employee}")

Total de registros: 39


Removendo duplicatas com base no nome e segmento do banco

In [15]:
# Remover duplicatas com base nas colunas 'Nome' e 'Segmento'
spark.sql("""
CREATE OR REPLACE TEMPORARY VIEW employee_temp_dedup AS
SELECT *
FROM (
    SELECT *, ROW_NUMBER() OVER (PARTITION BY Nome, Segmento ORDER BY Nome) AS row_num
    FROM employee_temp
) tmp
WHERE row_num = 1
""")
employee_dedup = spark.sql("SELECT * FROM employee_temp_dedup")
employee_dedup.show(truncate=False)

+-------------------------------+-------------+-------------+--------------+--------------+-------------------------------+------------------------+----------------+-----------------------------------------------------------------------------------+---------------------------+----------------------------------------------------------------------------------------------------------------+-----+-----------------+----------------------+-----------------+--------------+------------------------+-------------------------+---------------------------------+----------------------------------+--------+--------------------------------------------------+-------------+--------+-------+
|employer_name                  |reviews_count|culture_count|salaries_count|benefits_count|employer-website               |employer-headquarters   |employer-founded|employer-industry                                                                  |employer-revenue           |url                                      

In [16]:
total_records_employee_dedup = spark.sql("SELECT COUNT(*) AS total FROM employee_temp_dedup").collect()[0]['total']
print(f"Registros subtraídos: {total_records_employee - total_records_employee_dedup}")

Registros subtraídos: 2


Analisando o esquema dos dados

In [17]:
employee_temp_dedup = spark.sql("DESCRIBE employee_temp_dedup").show(truncate=False)

+----------------------------------+---------+-------+
|col_name                          |data_type|comment|
+----------------------------------+---------+-------+
|employer_name                     |string   |NULL   |
|reviews_count                     |int      |NULL   |
|culture_count                     |int      |NULL   |
|salaries_count                    |int      |NULL   |
|benefits_count                    |int      |NULL   |
|employer-website                  |string   |NULL   |
|employer-headquarters             |string   |NULL   |
|employer-founded                  |double   |NULL   |
|employer-industry                 |string   |NULL   |
|employer-revenue                  |string   |NULL   |
|url                               |string   |NULL   |
|Geral                             |double   |NULL   |
|Cultura e valores                 |double   |NULL   |
|Diversidade e inclusão            |double   |NULL   |
|Qualidade de vida                 |double   |NULL   |
|Alta lide

Salvando dados na camada _raw_ no formato parquet

In [18]:
drive_path = '/content/drive/MyDrive/USP/3 - Ingestão de dados/Atividade4/raw/employee'

os.makedirs(drive_path, exist_ok=True)

parquet_path = os.path.join(drive_path, 'employee_dedup.parquet')
csv_path = os.path.join(drive_path, 'employee_dedup.csv')

# Salvar o DataFrame como .parquet
employee_dedup.write.mode("overwrite").parquet(parquet_path)

# Salvar o DataFrame como .csv (com cabeçalho)
employee_dedup.write.mode("overwrite").option("header", "true").csv(csv_path)

print(f"Arquivos salvos em: {drive_path}")

Arquivos salvos em: /content/drive/MyDrive/USP/3 - Ingestão de dados/Atividade4/raw/employee


### **Claims file**

Localizando todos os arquivos contendo dados de reclamações

In [19]:
claim_dir = '/content/drive/MyDrive/USP/3 - Ingestão de dados/Atividade4/Fonte de Dados/Reclamações'
claim_files = os.listdir(claim_dir)
claim_paths = list(map(lambda file: os.path.join(claim_dir, file), claim_files))
claim_paths

['/content/drive/MyDrive/USP/3 - Ingestão de dados/Atividade4/Fonte de Dados/Reclamações/2022_tri_02_nao_ha_dados.csv',
 '/content/drive/MyDrive/USP/3 - Ingestão de dados/Atividade4/Fonte de Dados/Reclamações/2022_tri_03.csv',
 '/content/drive/MyDrive/USP/3 - Ingestão de dados/Atividade4/Fonte de Dados/Reclamações/2022_tri_01.csv',
 '/content/drive/MyDrive/USP/3 - Ingestão de dados/Atividade4/Fonte de Dados/Reclamações/2021_tri_02.csv',
 '/content/drive/MyDrive/USP/3 - Ingestão de dados/Atividade4/Fonte de Dados/Reclamações/2021_tri_04.csv',
 '/content/drive/MyDrive/USP/3 - Ingestão de dados/Atividade4/Fonte de Dados/Reclamações/2021_tri_03.csv',
 '/content/drive/MyDrive/USP/3 - Ingestão de dados/Atividade4/Fonte de Dados/Reclamações/2022_tri_04.csv',
 '/content/drive/MyDrive/USP/3 - Ingestão de dados/Atividade4/Fonte de Dados/Reclamações/2021_tri_01.csv']

In [20]:
claim_df = spark.read.csv(claim_paths, sep=';', encoding='latin1', header=True)
claim_df.createOrReplaceTempView("claim_temp")
records_claim = spark.sql("SELECT * FROM claim_temp")
records_claim.show(truncate=False)

+----+---------+------------------------------------------------------+----------------+--------+----------------------------------------------------------------+------+-----------------------------------------------+--------------------------------------------+---------------------------------------+-------------------------------+----------------------------------------+----------------------------+----------------------------+----+
|Ano |Trimestre|Categoria                                             |Tipo            |CNPJ IF |Instituição financeira                                          |Índice|Quantidade de reclamações reguladas procedentes|Quantidade de reclamações reguladas - outras|Quantidade de reclamações não reguladas|Quantidade total de reclamações|Quantidade total de clientes  CCS e SCR|Quantidade de clientes  CCS|Quantidade de clientes  SCR|_c14|
+----+---------+------------------------------------------------------+----------------+--------+-------------------------

Analisando o esquema dos dados

In [21]:
spark.sql("DESCRIBE claim_temp").show(truncate=False)

+-----------------------------------------------+---------+-------+
|col_name                                       |data_type|comment|
+-----------------------------------------------+---------+-------+
|Ano                                            |string   |NULL   |
|Trimestre                                      |string   |NULL   |
|Categoria                                      |string   |NULL   |
|Tipo                                           |string   |NULL   |
|CNPJ IF                                        |string   |NULL   |
|Instituição financeira                         |string   |NULL   |
|Índice                                         |string   |NULL   |
|Quantidade de reclamações reguladas procedentes|string   |NULL   |
|Quantidade de reclamações reguladas - outras   |string   |NULL   |
|Quantidade de reclamações não reguladas        |string   |NULL   |
|Quantidade total de reclamações                |string   |NULL   |
|Quantidade total de clientes  CCS e SCR       

In [22]:
spark.sql("SELECT * FROM claim_temp LIMIT 10").show()

+----+---------+--------------------+----------------+--------+----------------------+------+-----------------------------------------------+--------------------------------------------+---------------------------------------+-------------------------------+----------------------------------------+----------------------------+----------------------------+----+
| Ano|Trimestre|           Categoria|            Tipo| CNPJ IF|Instituição financeira|Índice|Quantidade de reclamações reguladas procedentes|Quantidade de reclamações reguladas - outras|Quantidade de reclamações não reguladas|Quantidade total de reclamações|Quantidade total de clientes  CCS e SCR|Quantidade de clientes  CCS|Quantidade de clientes  SCR|_c14|
+----+---------+--------------------+----------------+--------+----------------------+------+-----------------------------------------------+--------------------------------------------+---------------------------------------+-------------------------------+----------------

Removendo coluna desnecessária

In [23]:
# Remove _c14 -> comando EXCEPT e DROP COLUMN nao funciona
spark.sql("""
CREATE OR REPLACE TEMPORARY VIEW claim_temp_filtered AS
SELECT Ano, Trimestre, Categoria, Tipo, `CNPJ IF`, `Instituição financeira`,
`Índice`, `Quantidade de reclamações reguladas procedentes`, `Quantidade de reclamações reguladas - outras`,
`Quantidade de reclamações não reguladas`, `Quantidade total de reclamações`, `Quantidade total de clientes  CCS e SCR`,
`Quantidade de clientes  CCS`, `Quantidade de clientes  SCR`
FROM claim_temp
""")
spark.sql("DESCRIBE claim_temp_filtered").show(truncate=False)

+-----------------------------------------------+---------+-------+
|col_name                                       |data_type|comment|
+-----------------------------------------------+---------+-------+
|Ano                                            |string   |NULL   |
|Trimestre                                      |string   |NULL   |
|Categoria                                      |string   |NULL   |
|Tipo                                           |string   |NULL   |
|CNPJ IF                                        |string   |NULL   |
|Instituição financeira                         |string   |NULL   |
|Índice                                         |string   |NULL   |
|Quantidade de reclamações reguladas procedentes|string   |NULL   |
|Quantidade de reclamações reguladas - outras   |string   |NULL   |
|Quantidade de reclamações não reguladas        |string   |NULL   |
|Quantidade total de reclamações                |string   |NULL   |
|Quantidade total de clientes  CCS e SCR       

Contando a quantidade de linhas

In [24]:
total_records_claim= spark.sql("SELECT COUNT(*) AS total FROM claim_temp_filtered").collect()[0]['total']
print(f"Registros claim: {total_records_claim}")

Registros claim: 918


Salvando dados na camada _raw_ no formato parquet

In [25]:
claim_temp_filtered = spark.sql("SELECT * FROM claim_temp_filtered")

drive_path = '/content/drive/MyDrive/USP/3 - Ingestão de dados/Atividade4/raw/claim'

os.makedirs(drive_path, exist_ok=True)

parquet_path = os.path.join(drive_path, 'claim_temp_filtered.parquet')
csv_path = os.path.join(drive_path, 'claim_temp_filtered.csv')

# Salvar o DataFrame como .parquet
claim_temp_filtered.write.mode("overwrite").parquet(parquet_path)

# Salvar o DataFrame como .csv (com cabeçalho)
claim_temp_filtered.write.mode("overwrite").option("header", "true").csv(csv_path)

print(f"Arquivos salvos em: {drive_path}")

Arquivos salvos em: /content/drive/MyDrive/USP/3 - Ingestão de dados/Atividade4/raw/claim


---
## **Trusted**

### **Banks**

Aplicando transformações para melhorar a qualidade dos dados:

1 - Renomeando colunas do dataframe para ingles e no formato snake_case

2 - Para que os dados da coluna _cnpj_ estivessem de acordo com seu padrão, os valores incompletos receberam numerais zeros à esquerda até completar 8 dígitos

In [32]:

#drive
parquet_path = '/content/drive/MyDrive/USP/3 - Ingestão de dados/Atividade4/raw/banks/rwzd_bank.parquet'

# Registrar o arquivo .parquet como uma tabela temporária
spark.read.parquet(parquet_path).createOrReplaceTempView("rwzd_bank")
rwzd_bank = spark.sql("SELECT * FROM rwzd_bank")

# Executar uma consulta SQL na tabela temporária
spark.sql("""
CREATE OR REPLACE TEMP VIEW trzd_banks AS
SELECT Segmento AS segment, Nome AS financial_institution_name,
      CASE
           WHEN cnpj = '' THEN NULL
           ELSE LPAD(cnpj, 8, '0')
       END AS cnpj
FROM rwzd_bank
""")

# Mostrar a estrutura da tabela criada
spark.sql("DESCRIBE trzd_banks").show(truncate=False)


+--------------------------+---------+-------+
|col_name                  |data_type|comment|
+--------------------------+---------+-------+
|segment                   |string   |NULL   |
|financial_institution_name|string   |NULL   |
|cnpj                      |string   |NULL   |
+--------------------------+---------+-------+



In [33]:
spark.sql("SELECT * FROM trzd_banks LIMIT 100").show(truncate=False)

+-------+---------------------------------------------+--------+
|segment|financial_institution_name                   |cnpj    |
+-------+---------------------------------------------+--------+
|S1     |BANCO DO BRASIL - PRUDENCIAL                 |00000000|
|S1     |BRADESCO - PRUDENCIAL                        |60746948|
|S1     |BTG PACTUAL - PRUDENCIAL                     |30306294|
|S1     |CAIXA ECONOMICA FEDERAL - PRUDENCIAL         |00360305|
|S1     |ITAU - PRUDENCIAL                            |60872504|
|S1     |SANTANDER - PRUDENCIAL                       |90400888|
|S2     |BANRISUL - PRUDENCIAL                        |92702067|
|S2     |BANCO DO NORDESTE DO BRASIL S.A. - PRUDENCIAL|07237373|
|S2     |BNDES - PRUDENCIAL                           |33657248|
|S2     |CITIBANK - PRUDENCIAL                        |33479023|
|S2     |CREDIT SUISSE - PRUDENCIAL                   |33987793|
|S2     |SAFRA - PRUDENCIAL                           |58160789|
|S2     |VOTORANTIM - PRU

In [34]:
spark.sql("""
CREATE OR REPLACE TEMP VIEW trzd_bank_view AS
SELECT *,
       sha1(concat(cnpj, segment)) AS sk_cnpj_segment
FROM trzd_banks
""")

trzd_bank = spark.sql("SELECT * FROM trzd_bank_view")
trzd_bank.show(truncate=False)


+-------+---------------------------------------------+--------+----------------------------------------+
|segment|financial_institution_name                   |cnpj    |sk_cnpj_segment                         |
+-------+---------------------------------------------+--------+----------------------------------------+
|S1     |BANCO DO BRASIL - PRUDENCIAL                 |00000000|d9be4941c63af670d63086dcb0849a997e062a64|
|S1     |BRADESCO - PRUDENCIAL                        |60746948|2f59bebc86f6b4dd5628043cac240c6e6a6f651d|
|S1     |BTG PACTUAL - PRUDENCIAL                     |30306294|110ffbf347c0837827daf658f4f97950ebe9f741|
|S1     |CAIXA ECONOMICA FEDERAL - PRUDENCIAL         |00360305|587e4dca521e8c4426371a29658419ecb0087262|
|S1     |ITAU - PRUDENCIAL                            |60872504|4b6e5b20d113f3415704fd6a998f2e4500340b18|
|S1     |SANTANDER - PRUDENCIAL                       |90400888|75ddb1c713e8da4daab944522b8ee5dda29c2983|
|S2     |BANRISUL - PRUDENCIAL                

In [35]:
# Caminho no Google Drive para salvar os arquivos
drive_path = '/content/drive/MyDrive/USP/3 - Ingestão de dados/Atividade4/trusted'

# Criar a pasta se ela não existir
os.makedirs(drive_path, exist_ok=True)

# Caminho para salvar os arquivos
parquet_path = os.path.join(drive_path, 'trzd_bank.parquet')
csv_path = os.path.join(drive_path, 'trzd_bank.csv')

# Salvar o DataFrame como .parquet
rwzd_bank.write.mode("overwrite").parquet(parquet_path)

# Salvar o DataFrame como .csv (com cabeçalho)
rwzd_bank.write.mode("overwrite").option("header", "true").csv(csv_path)

print(f"Arquivos salvos em: {drive_path}")


Arquivos salvos em: /content/drive/MyDrive/USP/3 - Ingestão de dados/Atividade4/trusted


### **Employees**

In [50]:
# Drive da camada RAW
parquet_path = '/content/drive/MyDrive/USP/3 - Ingestão de dados/Atividade4/raw/employee/employee_dedup.parquet'

# Registrar o arquivo .parquet como uma tabela temporária
spark.read.parquet(parquet_path).createOrReplaceTempView("trusted_employee")


In [51]:
# Consulta a tabela de forma temporária para renomear as colunas e padronizar o cnpj
trusted_employee_rename = spark.sql("""CREATE OR REPLACE TEMPORARY VIEW trusted_employee_rename AS SELECT
    `employer-website` AS employer_website,
    `employer-headquarters` AS employer_headquarters,
    `employer-founded` AS employer_founded,
    `employer-industry` AS employer_industry,
    `employer-revenue` AS employer_revenue,
    `Geral` AS general_score,
    `Cultura e valores` AS culture_values_score,
    `Diversidade e inclusão` AS diversity_inclusion_score,
    `Qualidade de vida` AS life_quality_score,
    `Alta liderança` AS senior_leadership_score,
    `Remuneração e benefícios` AS compensation_benefits_score,
    `Oportunidades de carreira` AS career_opportunities_score,
    `Recomendam para outras pessoas(%)` AS recommendation_score,
    `Perspectiva positiva da empresa(%)` AS company_positive_score,
    Segmento AS segment,
    Nome AS financial_institution_name,
    case
        when cnpj = '' then NULL
        else LPAD(cnpj, 8, '0')
    end as cnpj
FROM trusted_employee
""")

spark.sql("DESCRIBE trusted_employee_rename").show(truncate=False)

+---------------------------+---------+-------+
|col_name                   |data_type|comment|
+---------------------------+---------+-------+
|employer_website           |string   |NULL   |
|employer_headquarters      |string   |NULL   |
|employer_founded           |double   |NULL   |
|employer_industry          |string   |NULL   |
|employer_revenue           |string   |NULL   |
|general_score              |double   |NULL   |
|culture_values_score       |double   |NULL   |
|diversity_inclusion_score  |double   |NULL   |
|life_quality_score         |double   |NULL   |
|senior_leadership_score    |double   |NULL   |
|compensation_benefits_score|double   |NULL   |
|career_opportunities_score |double   |NULL   |
|recommendation_score       |double   |NULL   |
|company_positive_score     |double   |NULL   |
|segment                    |string   |NULL   |
|financial_institution_name |string   |NULL   |
|cnpj                       |string   |NULL   |
+---------------------------+---------+-

In [52]:
spark.sql("SELECT * FROM trusted_employee_rename").show()

+--------------------+---------------------+----------------+--------------------+--------------------+-------------+--------------------+-------------------------+------------------+-----------------------+---------------------------+--------------------------+--------------------+----------------------+-------+--------------------------+--------+
|    employer_website|employer_headquarters|employer_founded|   employer_industry|    employer_revenue|general_score|culture_values_score|diversity_inclusion_score|life_quality_score|senior_leadership_score|compensation_benefits_score|career_opportunities_score|recommendation_score|company_positive_score|segment|financial_institution_name|    cnpj|
+--------------------+---------------------+----------------+--------------------+--------------------+-------------+--------------------+-------------------------+------------------+-----------------------+---------------------------+--------------------------+--------------------+---------------

Separando Estado e País da coluna employer-headquarters

In [53]:
spark.sql("""
CREATE OR REPLACE TEMPORARY VIEW trusted_employee_split AS
SELECT *,
       split(`employer_headquarters`, ',')[0] AS state_employer_headquarters,
       split(`employer_headquarters`, ',')[1] AS country_employer_headquarters
FROM trusted_employee_rename
""")

spark.sql("SELECT * FROM trusted_employee_split").show(truncate=False)

+-------------------------------+------------------------+----------------+-----------------------------------------------------------------------------------+---------------------------+-------------+--------------------+-------------------------+------------------+-----------------------+---------------------------+--------------------------+--------------------+----------------------+-------+--------------------------------------------------+--------+---------------------------+-----------------------------+
|employer_website               |employer_headquarters   |employer_founded|employer_industry                                                                  |employer_revenue           |general_score|culture_values_score|diversity_inclusion_score|life_quality_score|senior_leadership_score|compensation_benefits_score|career_opportunities_score|recommendation_score|company_positive_score|segment|financial_institution_name                        |cnpj    |state_employer_headquarte

Converter a coluna 'employer-founded' para inteiro, mantendo valores nulos

In [54]:
spark.sql("""
CREATE OR REPLACE TEMPORARY VIEW trusted_employee_cast AS
SELECT *,
       CAST(employer_founded AS INT) AS employer_founded_correct
FROM trusted_employee_split
""")
spark.sql("SELECT * FROM trusted_employee_cast").show(truncate=False)

+-------------------------------+------------------------+----------------+-----------------------------------------------------------------------------------+---------------------------+-------------+--------------------+-------------------------+------------------+-----------------------+---------------------------+--------------------------+--------------------+----------------------+-------+--------------------------------------------------+--------+---------------------------+-----------------------------+------------------------+
|employer_website               |employer_headquarters   |employer_founded|employer_industry                                                                  |employer_revenue           |general_score|culture_values_score|diversity_inclusion_score|life_quality_score|senior_leadership_score|compensation_benefits_score|career_opportunities_score|recommendation_score|company_positive_score|segment|financial_institution_name                        |cnpj    |

In [55]:
trusted_employee_cast = spark.sql("SELECT * FROM trusted_employee_cast")
# Caminho no Google Drive para salvar os arquivos
drive_path = '/content/drive/MyDrive/USP/3 - Ingestão de dados/Atividade4/trusted'

# Criar a pasta se ela não existir
os.makedirs(drive_path, exist_ok=True)

# Caminho para salvar os arquivos
parquet_path = os.path.join(drive_path, 'employee.parquet')
csv_path = os.path.join(drive_path, 'employee.csv')

# Salvar o DataFrame como .parquet
trusted_employee_cast.write.mode("overwrite").parquet(parquet_path)

# Salvar o DataFrame como .csv (com cabeçalho)
trusted_employee_cast.write.mode("overwrite").option("header", "true").csv(csv_path)

print(f"Arquivos salvos em: {drive_path}")


Arquivos salvos em: /content/drive/MyDrive/USP/3 - Ingestão de dados/Atividade4/trusted


### **Claims**

In [58]:
# Drive da camada RAW
parquet_path = '/content/drive/MyDrive/USP/3 - Ingestão de dados/Atividade4/raw/claim/claim_temp_filtered.parquet'

# Registrar o arquivo .parquet como uma tabela temporária
spark.read.parquet(parquet_path).createOrReplaceTempView("trusted_claim")
trusted_claim = spark.sql("SELECT * FROM trusted_claim")
trusted_claim.show(truncate=False)


+----+---------+------------------------------------------------------+----------------+--------+----------------------------------------------------------------+------+-----------------------------------------------+--------------------------------------------+---------------------------------------+-------------------------------+----------------------------------------+----------------------------+----------------------------+
|Ano |Trimestre|Categoria                                             |Tipo            |CNPJ IF |Instituição financeira                                          |Índice|Quantidade de reclamações reguladas procedentes|Quantidade de reclamações reguladas - outras|Quantidade de reclamações não reguladas|Quantidade total de reclamações|Quantidade total de clientes  CCS e SCR|Quantidade de clientes  CCS|Quantidade de clientes  SCR|
+----+---------+------------------------------------------------------+----------------+--------+-----------------------------------

In [59]:
spark.sql("""
CREATE OR REPLACE TEMPORARY VIEW trusted_claim_view AS
SELECT Ano AS year_claim,
       Trimestre AS quarter_claim,
       Categoria AS category,
       Tipo AS bank_type,
       `CNPJ IF` AS cnpj,
       `Instituição financeira` AS financial_institution_name,
       `Índice` AS bank_index,
       `Quantidade de reclamações reguladas procedentes` AS number_of_regulated_proceeding_complaints,
       `Quantidade de reclamações reguladas - outras` AS number_of_regulated_other_complaints,
       `Quantidade de reclamações não reguladas` AS number_of_non_regulated_complaints,
       `Quantidade total de reclamações` AS total_number_of_complaints,
       `Quantidade total de clientes  CCS e SCR` AS total_number_of_customers,
       `Quantidade de clientes  CCS` AS number_of_ccs_customers,
       `Quantidade de clientes  SCR` AS number_of_scr_customers
FROM trusted_claim
""")
trzd_claim = spark.sql("SELECT * FROM trusted_claim_view")
trzd_claim.show(truncate=False)


+----------+-------------+------------------------------------------------------+----------------+--------+----------------------------------------------------------------+----------+-----------------------------------------+------------------------------------+----------------------------------+--------------------------+-------------------------+-----------------------+-----------------------+
|year_claim|quarter_claim|category                                              |bank_type       |cnpj    |financial_institution_name                                      |bank_index|number_of_regulated_proceeding_complaints|number_of_regulated_other_complaints|number_of_non_regulated_complaints|total_number_of_complaints|total_number_of_customers|number_of_ccs_customers|number_of_scr_customers|
+----------+-------------+------------------------------------------------------+----------------+--------+----------------------------------------------------------------+----------+-------------------

In [60]:
spark.sql("""
CREATE OR REPLACE TEMPORARY VIEW trusted_claim_view_transformed AS
SELECT year_claim,
       regexp_replace(quarter_claim, 'º', '') AS quarter_claim,
       category,
       bank_type,
       CASE
           WHEN cnpj = ' ' THEN NULL
           ELSE LPAD(cnpj, 8, '0')
       END AS cnpj,
       financial_institution_name,
       CAST(regexp_replace(regexp_replace(regexp_replace(`bank_index`, r'\.', ''), r',', '.'), r' ', '') AS FLOAT) AS bank_index,
       number_of_regulated_proceeding_complaints,
       number_of_regulated_other_complaints,
       number_of_non_regulated_complaints,
       total_number_of_complaints,
       total_number_of_customers,
       number_of_ccs_customers,
       number_of_scr_customers
FROM trusted_claim_view
""")
trzd_claim = spark.sql("SELECT * FROM trzd_claim_view_transformed")
trzd_claim.show(truncate=False)

+-------+---------------------------------------------+--------+
|segment|financial_institution_name                   |cnpj    |
+-------+---------------------------------------------+--------+
|S1     |BANCO DO BRASIL - PRUDENCIAL                 |00000000|
|S1     |BRADESCO - PRUDENCIAL                        |60746948|
|S1     |BTG PACTUAL - PRUDENCIAL                     |30306294|
|S1     |CAIXA ECONOMICA FEDERAL - PRUDENCIAL         |00360305|
|S1     |ITAU - PRUDENCIAL                            |60872504|
|S1     |SANTANDER - PRUDENCIAL                       |90400888|
|S2     |BANRISUL - PRUDENCIAL                        |92702067|
|S2     |BANCO DO NORDESTE DO BRASIL S.A. - PRUDENCIAL|07237373|
|S2     |BNDES - PRUDENCIAL                           |33657248|
|S2     |CITIBANK - PRUDENCIAL                        |33479023|
|S2     |CREDIT SUISSE - PRUDENCIAL                   |33987793|
|S2     |SAFRA - PRUDENCIAL                           |58160789|
|S2     |VOTORANTIM - PRU

In [62]:
spark.sql("""
CREATE OR REPLACE TEMPORARY VIEW trusted_claim_view_transformed2 AS
SELECT year_claim,
       quarter_claim,
       category,
       bank_type,
       cnpj,
       financial_institution_name,
       bank_index,
       number_of_regulated_proceeding_complaints,
       number_of_regulated_other_complaints,
       number_of_non_regulated_complaints,
       total_number_of_complaints,
       CAST(regexp_replace(total_number_of_customers, r' ', '') AS INT) AS total_number_of_customers,
       CAST(regexp_replace(`number_of_ccs_customers`, r' ', '') AS INT) AS number_of_ccs_customers,
       CAST(regexp_replace(`number_of_scr_customers`, r' ', '') AS INT) AS number_of_scr_customers
FROM trusted_claim_view_transformed
""")
trzd_claim = spark.sql("SELECT * FROM trusted_claim_view_transformed2")
trzd_claim.show(truncate=False)


+----------+-------------+------------------------------------------------------+----------------+--------+----------------------------------------------------------------+----------+-----------------------------------------+------------------------------------+----------------------------------+--------------------------+-------------------------+-----------------------+-----------------------+
|year_claim|quarter_claim|category                                              |bank_type       |cnpj    |financial_institution_name                                      |bank_index|number_of_regulated_proceeding_complaints|number_of_regulated_other_complaints|number_of_non_regulated_complaints|total_number_of_complaints|total_number_of_customers|number_of_ccs_customers|number_of_scr_customers|
+----------+-------------+------------------------------------------------------+----------------+--------+----------------------------------------------------------------+----------+-------------------

---
## **Delivery**